In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/baseline-model/hacktest.csv
/kaggle/input/baseline-model/hacktrain.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:
# Logistic Regression with Feature Scaling & Polynomial Features
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

# Load data
train = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
test = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")

# Drop unnecessary column if exists
for df in [train, test]:
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

# Store test IDs for submission
test_ids = test['ID']

# Encode labels
y = train['class']
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(y)

# Drop ID and target from features
X = train.drop(columns=['ID', 'class'])
X_test = test.drop(columns=['ID'])

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
X_test_imputed = imputer.transform(X_test)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Add polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Train Logistic Regression
clf = LogisticRegression(max_iter=2000, solver='lbfgs', multi_class='multinomial')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(clf, X_poly, y_encoded, cv=skf, scoring='accuracy')
print("CV Accuracy Scores:", scores)
print("Mean Accuracy:", scores.mean())

# Fit on full data and predict
clf.fit(X_poly, y_encoded)
preds = clf.predict(X_test_poly)
pred_labels = label_enc.inverse_transform(preds)

# Save submission
submission = pd.DataFrame({
    'ID': test_ids,
    'class': pred_labels
})
submission.to_csv("submission_poly.csv", index=False)
print("✅ Second submission saved as submission_poly.csv")


CV Accuracy Scores: [0.9325   0.9375   0.930625 0.923125 0.9275  ]
Mean Accuracy: 0.93025
✅ Second submission saved as submission_poly.csv


In [3]:
!ls -lh /kaggle/working/


total 36K
-rw-r--r-- 1 root root 33K Jun 19 06:22 submission_poly.csv
